# To Visualize and Analysze tomorrow

In [44]:
import numpy as np
import tensorflow as tf
import os
import cv2
import pandas as pd
SEED = 1234

In [41]:
# Creating the training set
dataset_dir = os.getcwd() # Obtain 
dataset_dir = os.path.join(dataset_dir,'MaskDataset')
print(dataset_dir)
train_dir = os.path.join(dataset_dir, 'training')

classes = ["nomask","allmask","someone"]

# Create generators to read images from dataset directory
# -------------------------------------------------------

# Batch size
bs = 8

# img shape
img_h = 256
img_w = 256

num_classes = 3

# for image in os.path.listdir():  # all the images in the folder



c:\Users\enric\Downloads\AN2DL-1st-Project\MaskDataset


## Importing the Json file and creating a dataframe

In [56]:

import json
with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic = json.load(f)

dataframe = pd.DataFrame(dic.items())  # putting all the 
dataframe.rename(columns = {0:'filename', 1:'class'}, inplace = True)
dataframe["class"] = dataframe["class"].astype(str)


## Creating the Training Set

In [69]:
# Creating the training set for the images ! 
train_gen = []
train_gen = train_data_gen.flow_from_dataframe(dataframe,
                                               training_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED) 

Found 5614 validated image filenames belonging to 3 classes.


In [68]:
# Test  
test_dir = os.path.join(dataset_dir, 'test')
print(test_dir)
test_gen = []
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             batch_size=bs, 
                                             classes=classes,
                                             class_mode='categorical',
                                             shuffle=False,
                                             seed=SEED)

c:\Users\enric\Downloads\AN2DL-1st-Project\MaskDataset\test
Found 0 images belonging to 3 classes.


In [64]:
# Check how keras assigned the labels
train_gen.class_indices

{'0': 0, '1': 1, '2': 2}

In [70]:
# Create Dataset objects
# ----------------------

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Shuffle (Already done in generator..)
# train_dataset          train_dataset.shuffle(buffer_size=len(train_gen))

# Normalize images (Already done in generator..)
# def normalize_img(x_, y_):
#     return tf.cast(x_, tf.float32) / 255., y_

# train_dataset = train_dataset.map(normalize_img)

# 1-hot encoding <- for categorical cross entropy (Already done in generator..)
# def to_categorical(x_, y_):
#     return x_, tf.one_hot(y_, depth=10)

# train_dataset = train_dataset.map(to_categorical)

# Divide in batches (Already done in generator..)
# train_dataset = train_dataset.batch(bs)

# Repeat
# Without calling the repeat function the dataset 
# will be empty after consuming all the images
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
valid_dataset = valid_dataset.repeat()

# Test
# ----
test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, img_h, img_w, 3], [None, num_classes]))

# Repeat
test_dataset = valid_dataset.repeat()

In [71]:
# Architecture: Features extraction -> Classifier

start_f = 8
depth = 5

model = tf.keras.Sequential()

# Features extraction
for i in range(depth):

    if i == 0:
        input_shape = [img_h, img_w, 3]
    else:
        input_shape=[None]

    # Conv block: Conv2D -> Activation -> Pooling
    model.add(tf.keras.layers.Conv2D(filters=start_f, 
                                     kernel_size=(3, 3),
                                     strides=(1, 1),
                                     padding='same',
                                     input_shape=input_shape))
    model.add(tf.keras.layers.ReLU())
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

    start_f *= 2
    
# Classifier
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [72]:
# Visualize created model as a table
model.summary()

# Visualize initialized weights
model.weights

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 8)       224       
_________________________________________________________________
re_lu (ReLU)                 (None, 256, 256, 8)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 8)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 16)      1168      
_________________________________________________________________
re_lu_1 (ReLU)               (None, 128, 128, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        4

,
          [-0.06297   ,  0.02729859, -0.0417695 , ...,  0.10523082,
           -0.10890945, -0.06804118],
          [-0.06471072,  0.04734232, -0.077664  , ..., -0.00634678,
            0.10633209,  0.05524176],
          ...,
          [ 0.1035709 , -0.11415755,  0.11062964, ..., -0.10747522,
            0.04693709,  0.08927701],
          [ 0.03105039,  0.00433969, -0.07344852, ...,  0.09285342,
           -0.00319485,  0.04962403],
          [ 0.01181971, -0.01264034, -0.02193139, ..., -0.04008755,
           -0.00900841, -0.05312627]]]], dtype=float32)>,
 <tf.Variable 'conv2d_2/bias:0' shape=(32,) dtype=float32, numpy=
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       dtype=float32)>,
 <tf.Variable 'conv2d_3/kernel:0' shape=(3, 3, 32, 64) dtype=float32, numpy=
 array([[[[-4.40888815e-02, -4.94686365e-02, -4.62609753e-02, ...,
            1.36670247e-02,  7.15393499e-02, -2.182915

In [73]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [76]:
model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_gen))

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

ValueError: When passing an infinitely repeating dataset, you must specify the `validation_steps` argument.

In [2]:
def create_csv(results, results_dir='./'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')

In [4]:
dataset_dir = "C:/Users/enric/Downloads/AN2DL-1st-Project/MaskDataset"
test_dir = os.path.join(dataset_dir, 'test')

images = [f for f in listdir(test_dir)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

test_gen = train_data_gen.flow_from_dataframe(images,
                                               test_dir,
                                               batch_size=bs,
                                               target_size=(img_h, img_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)


test_gen.reset()

predictions = model.predict_generator(test_gen, len(test_gen), verbose=1)

results = {}
images = test_gen.filenames
i = 0



for p in predictions:
  prediction = np.argmax(p)
  import ntpath
  image_name = ntpath.basename(images[i])
  results[image_name] = str(prediction)
  i = i + 1

NameError: name 'listdir' is not defined